In [ ]:
import pandas as pd
import numpy as np

from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor

In [ ]:
sld= pd.read_parquet('data/Smart_Location_Database.parquet')
sld['FIPS'] = sld['STATEFP'].astype('str').str.zfill(2) + sld['COUNTYFP'].astype('str').str.zfill(3)

In [ ]:
for col in sld.columns:
    if -99999 in sld[col].values:
        print(col)

D4A
D4C
D4D
D4E
D5BR
D5BE
D5DR
D5DRI
D5DE
D5DEI


In [ ]:
## Additional Cleaning

# All CBGs with population-weighted centroids that were further than three-quarter miles from a transit stop were assigned a value of “-99999.
sld['D4A'] =sld['D4D'].apply(lambda x: np.nan if x == -99999 else x)

# CBGs in areas that do not have transit service were assigned the value “-99999.”
sld['D4C'] =sld['D4D'].apply(lambda x: 0 if x == -99999 else x)

# CBGs in areas that did not have transit service were given the value “-99999.”
sld['D4D'] =sld['D4D'].apply(lambda x: 0 if x == -99999 else x)

# All CBGs in areas where GTFS service data were unavailable were assigned a value of “-99999.
sld['D4E'] =sld['D4E'].apply(lambda x: 0 if x == -99999 else x)


sld['D5BR'] =sld['D5BR'].apply(lambda x: np.nan if x == -99999 else x)
sld['D5BE'] =sld['D5BE'].apply(lambda x: np.nan if x == -99999 else x)
sld['D5DR'] =sld['D5DR'].apply(lambda x: np.nan if x == -99999 else x)
sld['D5DRI'] =sld['D5DRI'].apply(lambda x: np.nan if x == -99999 else x)
sld['D5DE'] =sld['D5DE'].apply(lambda x: np.nan if x == -99999 else x)
sld['D5DEI'] =sld['D5DEI'].apply(lambda x: np.nan if x == -99999 else x)


In [ ]:
# drop NAs for now, comment out when imputation is available
sld.dropna(inplace = True)

## Split Dataset

In [ ]:
num_unique_state_county_FIPS = len(sld['FIPS'].unique())
train_frac = int(num_unique_state_county_FIPS*.80)

train_FIPS = np.random.choice(sld['FIPS'].unique(), size= train_frac, replace=False)


In [ ]:

train_df = sld[sld.FIPS.isin(train_FIPS )]
test_df = sld[sld.FIPS.isin(train_FIPS )== False]

In [ ]:
#list(train_df.columns)

## Candidate Features

In [ ]:
features =['CBSA_POP','CBSA_EMP','CBSA_WRK','Ac_Total',
 'Ac_Water',
 'Ac_Land',
 'Ac_Unpr',
 'TotPop',
 'CountHU',
 'HH',
 'P_WrkAge',
 'AutoOwn0',
 'Pct_AO0',
 'AutoOwn1',
 'Pct_AO1',
 'AutoOwn2p',
 'Pct_AO2p',
 'Workers',
 'R_LowWageWk',
 'R_MedWageWk',
 'R_HiWageWk',
 'R_PCTLOWWAGE',
 'TotEmp',
 'E5_Ret',
 'E5_Off',
 'E5_Ind',
 'E5_Svc',
 'E5_Ent',
 'E8_Ret',
 'E8_off',
 'E8_Ind',
 'E8_Svc',
 'E8_Ent',
 'E8_Ed',
 'E8_Hlth',
 'E8_Pub',
 'E_LowWageWk',
 'E_MedWageWk',
 'E_HiWageWk',
 'E_PctLowWage',
 'D1A',
 'D1B',
 'D1C',
 'D1C5_RET',
 'D1C5_OFF',
 'D1C5_IND',
 'D1C5_SVC',
 'D1C5_ENT',
 'D1C8_RET',
 'D1C8_OFF',
 'D1C8_IND',
 'D1C8_SVC',
 'D1C8_ENT',
 'D1C8_ED',
 'D1C8_HLTH',
 'D1C8_PUB',
 'D1D',
 'D1_FLAG',
 'D2A_JPHH',
 'D2B_E5MIX',
 'D2B_E5MIXA',
 'D2B_E8MIX',
 'D2B_E8MIXA',
 'D2A_EPHHM',
 'D2C_TRPMX1',
 'D2C_TRPMX2',
 'D2C_TRIPEQ',
 'D2R_JOBPOP',
 'D2R_WRKEMP',
 'D2A_WRKEMP',
 'D2C_WREMLX',
 'D3A',
 'D3AAO',
 'D3AMM',
 'D3APO',
 'D3B',
 'D3BAO',
 'D3BMM3',
 'D3BMM4',
 'D3BPO3',
 'D3BPO4',
 'D4B025',
 'D4B050',
 'D4C',
 'D4D',
 'D4E',
 'D5AE',
 'D5BR',
 'D5BE',
 'D5CR',
 'D5CRI',
 'D5CE',
 'D5CEI',
 'D5DR',
 'D5DRI',
 'D5DE',
 'D5DEI',
 'Households',
 'Workers_1',
 'Residents',
 'Drivers',
 'Vehicles',
 'White',
 'Male',
 'Lowwage',
 'Medwage',
 'Highwage',
 'W_P_Lowwage',
 'W_P_Medwage',
 'W_P_Highwage',
 'GasPrice',
 'logd1a',
 'logd1c',
 'logd3aao',
 'logd3apo',
 'd4bo25',
 'd5dei_1',
 'logd4d',
 'UPTpercap',
 'B_C_constant',
 'B_C_male',
 'B_C_ld1c',
 'B_C_drvmveh',
 'B_C_ld1a',
 'B_C_ld3apo',
 'B_C_inc1',
 'B_C_gasp',
 'B_N_constant',
 'B_N_inc2',
 'B_N_inc3',
 'B_N_white',
 'B_N_male',
 'B_N_drvmveh',
 'B_N_gasp',
 'B_N_ld1a',
 'B_N_ld1c',
 'B_N_ld3aao',
 'B_N_ld3apo',
 'B_N_d4bo25',
 'B_N_d5dei',
 'B_N_UPTpc',
 'C_R_Households',
 'C_R_Pop',
 'C_R_Workers',
 'C_R_Drivers',
 'C_R_Vehicles',
 'C_R_White',
 'C_R_Male',
 'C_R_Lowwage',
 'C_R_Medwage',
 'C_R_Highwage',
 'C_R_DrmV',
 'NonCom_VMT_Per_Worker',
 'Com_VMT_Per_Worker',
 'VMT_per_worker',
 'VMT_tot_min',
 'VMT_tot_max',
 'VMT_tot_avg',
 'GHG_per_worker',
 'Annual_GHG',
 'SLC_score',
 'Shape__Area',
 'Shape__Length']

# scores of metropolitan regions, such as walkability, access to transit, and destination accessibility
targets = ['NatWalkInd','D4A','D5AR']

In [ ]:
len(features)

163

In [ ]:
y_train = train_df['NatWalkInd']
y_test = test_df['NatWalkInd']

In [ ]:
len(y_train)

56012

In [ ]:
# Scaling Dataset

scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(train_df[features])
X_test_scaled = scaler.transform(test_df[features])

In [ ]:
X_train_scaled

array([[3.71517951e-01, 3.86730676e-01, 3.75576743e-01, ...,
        7.74509558e-01, 4.91833731e-04, 1.57031100e-02],
       [3.71517951e-01, 3.86730676e-01, 3.75576743e-01, ...,
        7.06877593e-01, 1.67427238e-04, 7.62324695e-03],
       [3.71517951e-01, 3.86730676e-01, 3.75576743e-01, ...,
        7.37397295e-01, 1.15352653e-03, 1.92569496e-02],
       ...,
       [7.79697662e-03, 1.00395092e-02, 8.82269105e-03, ...,
        6.54595310e-01, 1.75959201e-03, 3.61269794e-02],
       [8.06494361e-02, 9.29904006e-02, 8.64399089e-02, ...,
        6.84009868e-01, 5.09661069e-04, 1.03582819e-02],
       [8.06494361e-02, 9.29904006e-02, 8.64399089e-02, ...,
        6.48117513e-01, 1.39836184e-03, 2.87415827e-02]])

Lasso Regression (L1-based feature selection)

In [ ]:
linlasso = Lasso(max_iter = 10000).fit(X_train_scaled, y_train)

In [ ]:
print('lasso regression linear model coeff:\n{}'.format(linlasso.coef_))
print('Nonzero features: \n{}'.format(np.sum(linlasso.coef_ != 0)))
print('R-squared score (training): {:.3f}'.format(linlasso.score(X_train_scaled, y_train)))
print('R-squared score (test): {:.3f}'.format(linlasso.score(X_test_scaled, y_test)))

lasso regression linear model coeff:
[ 0.  0.  0. -0. -0. -0. -0. -0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0.
 -0. -0.  0. -0.  0.  0.  0. -0.  0.  0.  0.  0. -0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. -0.  0.  0.  0.  0.  0.  0. -0. -0. -0. -0.  0. -0. -0. -0.  0. -0.
 -0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -0. -0. -0.  0. -0.
 -0. -0.  0.  0. -0.  0. -0.  0. -0. -0.  0.  0.  0.  0. -0.  0.  0.  0.
  0.  0. -0.  0. -0. -0.  0.  0. -0. -0. -0. -0.  0. -0. -0. -0.  0. -0.
 -0.]
Nonzero features: 
0
R-squared score (training): 0.000
R-squared score (test): -0.019


In [ ]:
print('features:')
for e in sorted(list(zip(list(train_df[features]), linlasso.coef_)), key=lambda e: -abs(e[1])):
    if e[1] != 0:
        print("\t{}, {:.3f}".format(e[0], e[1]))


print('target: D4A')

features:
target: D4A


Random Forest Regressor

In [ ]:
rfc = RandomForestRegressor(n_jobs = -1, random_state = 42)

rfc.fit(train_df[features], y_train)
    
_ = [i for i in zip(features,rfc.feature_importances_)]

top_x = sorted(_, key = lambda x : x[1],reverse=True)
score = rfc.score(test_df[features],y_test)

In [ ]:
print(score)

0.9254973063361305


In [ ]:
print(top_x)

[('D3B', 0.4043106419200125), ('D2B_E8MIXA', 0.19083901531222), ('D2C_TRPMX1', 0.12743161386750837), ('D2A_EPHHM', 0.11001591203674088), ('D1C', 0.03872190667756734), ('logd1c', 0.03201337360762771), ('logd4d', 0.005002404580292401), ('D4D', 0.0024811060785004183), ('D4C', 0.0024420242625628275), ('D4E', 0.0024033420716290236), ('D2B_E8MIX', 0.002145582996386012), ('AutoOwn2p', 0.0018925382474719804), ('D3BPO3', 0.0015642118406503569), ('Pct_AO0', 0.0015211235942095027), ('D3BPO4', 0.0015166669367977046), ('D2A_JPHH', 0.001366872493360188), ('D5DRI', 0.0012837601106902004), ('Pct_AO2p', 0.0012758856997701327), ('D3A', 0.0012372237258305645), ('D5DEI', 0.001213824759428787), ('Vehicles', 0.001203894480279909), ('D3BMM4', 0.0011208477933560843), ('D2B_E5MIX', 0.001101768663012263), ('NonCom_VMT_Per_Worker', 0.0010755913777522948), ('D5BR', 0.0010615941235224496), ('D2C_TRPMX2', 0.0010336053631779527), ('D5CRI', 0.0010297547551569007), ('D5CEI', 0.0009894460513496055), ('D1D', 0.000964956

In [ ]:
top_x[:10]

[('D3B', 0.4043106419200125),
 ('D2B_E8MIXA', 0.19083901531222),
 ('D2C_TRPMX1', 0.12743161386750837),
 ('D2A_EPHHM', 0.11001591203674088),
 ('D1C', 0.03872190667756734),
 ('logd1c', 0.03201337360762771),
 ('logd4d', 0.005002404580292401),
 ('D4D', 0.0024811060785004183),
 ('D4C', 0.0024420242625628275),
 ('D4E', 0.0024033420716290236)]

Walkability index comprised of weighted sum of the ranked values of [D2a_EpHHm] (D2A_Ranked),[D2b_E8MixA] (D2B_Ranked), [D3b] (D3B_Ranked) and [D4a] (D4A_Ranked)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e75edf0e-32f1-42b8-8a27-9dc0078a206d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>